In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings

pdf_data_path = 'C:\Project\data'
vector_db_path = 'vectorstore/db_history'

def create_db_from_file():
    loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls = PyPDFLoader)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 50)
    chunks = text_splitter.split_documents(documents)

    embedding_model = GPT4AllEmbeddings(model_file = 'models/PhoGPT-4B-Chat-Q4_K_M.gguf')
    db = FAISS.from_documents(chunks, embedding= embedding_model)
    db.save_local(vector_db_path)
    return db

create_db_from_file()

In [ ]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS

# Cấu hình
model_file = "C:/Project/models/PhoGPT-4B-Chat-Q4_K_M.gguf"
vector_db_path = "vectorstore/db_history"

# Load LLM
def llm_load(model_path): 
    llm = LlamaCpp(
        model_path=model_path,
        temperature=0.01,
        n_gpu_layers=20,
        max_new_tokens=256,
        n_ctx=1024  # nếu model hỗ trợ
    )
    return llm

# Load vector DB và truy vấn BM25
def get_context_from_bm25(question, k=2):
    embedding_model = GPT4AllEmbeddings(model_file=model_file)
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)

    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(question)
    
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

llm = llm_load(model_file)
# Tạo prompt
def build_prompt(context, question):
    template = """[Vai trò]
                  Bạn là một trợ lý AI thông minh, am hiểu sâu sắc về lịch sử Việt Nam và thế giới, đặc biệt là các giai đoạn lịch sử quan trọng như thời kỳ phong kiến, thời kỳ thuộc địa, chiến tranh và hiện đại.
                  Khi người dùng đặt câu hỏi liên quan đến lịch sử, hãy ưu tiên phân tích đúng ngữ cảnh, nêu rõ nguyên nhân – kết quả, và cung cấp thông tin chính xác dựa trên tài liệu lịch sử chính thống.
                  Nếu người dùng đặt một câu hỏi **không liên quan đến lịch sử**, ví dụ về khoa học, đời sống, hoặc chính bạn, bạn vẫn có thể trả lời một cách tự nhiên, thông minh và thân thiện – miễn là câu hỏi hợp lý và không yêu cầu thông tin vượt giới hạn.
                  Nếu câu hỏi mơ hồ hoặc thiếu thông tin, hãy lịch sự yêu cầu người dùng làm rõ thêm để có thể giúp đỡ tốt hơn.

                Bối cảnh (nếu có):  {context}
                Câu hỏi:  {question}

                [Format Output]

                - Dựa vào thông tin lịch sử chính xác
                - Trình bày dễ hiểu, dành cho người phổ thông
                - Có thể trích dẫn mốc thời gian, nhân vật lịch sử, sự kiện liên quan
                - Nếu không đủ thông tin, hãy nói rõ rằng bạn không chắc chắn hoặc gợi ý người dùng kiểm tra thêm tài liệu

                Trả lời:"""
    prompt = PromptTemplate.from_template(template)
    return prompt.format(context=context, question=question)
# Thực thi với câu hỏi và lấy phản hồi từ vector DB hoặc LLM trực tiếp
def handle_question(question):
    context = get_context_from_bm25(question, k=3)
    if context:  # Nếu có dữ liệu trả về từ vector DB
        final_prompt = build_prompt(context, question)
        response = llm.invoke(final_prompt)
    else:
        # Nếu không có dữ liệu trả về từ vector DB, trả lời bằng LLM
        prompt = f"Bạn là một trợ lý AI thông minh. Câu hỏi: {question}."
        response = llm.invoke(prompt)
    return response
# Thực thi
question = "Sao bạn trả lời vị vua đầu tiên của Việt Nam là Đinh Tiên Hoàng?"
context = get_context_from_bm25(question, k=3)
print(context)
final_prompt = build_prompt(context, question)

response = handle_question(question)
print(response)



In [ ]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS

# --- Cấu hình ---
model_file = "C:/Project/models/PhoGPT-4B-Chat-Q4_K_M.gguf"
vector_db_path = "vectorstore/db_history"

# --- Load mô hình LLM ---
def llm_load(model_path): 
    llm = LlamaCpp(
        model_path=model_path,
        temperature=0.01,
        n_gpu_layers=20,
        max_new_tokens=512,
        n_ctx=2048  # Tùy model
    )
    return llm

# --- Tải vector DB và tìm tài liệu liên quan ---
def get_context_from_bm25(question, k=3):
    embedding_model = GPT4AllEmbeddings(model_file=model_file)
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

# --- Prompt Template chính ---
def build_prompt(context, question):
    template = """[Vai trò]
Bạn là một trợ lý AI thông minh, am hiểu sâu sắc về lịch sử Việt Nam và thế giới, đặc biệt là các giai đoạn lịch sử quan trọng như thời kỳ phong kiến, thời kỳ thuộc địa, chiến tranh và hiện đại.

Khi người dùng đặt câu hỏi liên quan đến lịch sử, hãy ưu tiên phân tích đúng ngữ cảnh, nêu rõ nguyên nhân – kết quả, và cung cấp thông tin chính xác dựa trên tài liệu lịch sử chính thống.

Nếu người dùng đặt một câu hỏi **không liên quan đến lịch sử**, ví dụ về khoa học, đời sống, hoặc chính bạn, bạn vẫn có thể trả lời một cách tự nhiên, thông minh và thân thiện – miễn là câu hỏi hợp lý và không yêu cầu thông tin vượt giới hạn.

Nếu câu hỏi mơ hồ hoặc thiếu thông tin, hãy lịch sự yêu cầu người dùng làm rõ thêm để có thể giúp đỡ tốt hơn.

---

Bối cảnh (nếu có):  
{context}

Câu hỏi:  
{question}

---

[Format Output]

- Dựa vào thông tin chính xác
- Trình bày rõ ràng, dễ hiểu
- Trích dẫn mốc thời gian, nhân vật, sự kiện (nếu có)
- Nếu không đủ thông tin, hãy nói rõ rằng bạn chưa chắc chắn hoặc gợi ý người dùng tìm thêm tài liệu

Trả lời:
"""
    prompt = PromptTemplate.from_template(template)
    return prompt.format(context=context, question=question)

# --- Trả lời câu hỏi chính ---
def handle_question(llm, question):
    context = get_context_from_bm25(question, k=3)
    prompt = build_prompt(context or "Không có tài liệu lịch sử phù hợp.", question)
    response = llm.invoke(prompt)
    return response

# --- Chạy ---
if __name__ == "__main__":
    llm = llm_load(model_file)

    while True:
        question = input("\n❓ Bạn muốn hỏi gì? (gõ 'exit' để thoát): ")
        if question.lower() in ["exit", "quit"]:
            break

        response = handle_question(llm, question)
        print("\n🤖 Trả lời:\n", response)
